## Introduction

**Using the "Hash-Trick": Calculate Fingerprints for Molecules and build a neural network for prediction of the binding affinity.**

Link to the competition and data: https://www.kaggle.com/competitions/leash-BELKA/overview

**Finding a good topology for the net still poses a big challenge.**

**Since the dataset is too big, we need to provide a data stream.
I thought it might be better to prepare TFRecord-Files to speed up the data supply.
But the data is even too much for my 20GB disc memory on Kaggle. So using a datastream from the provided files may be the only option to train on all of the data.**

**One can try to use TPUs instead of CPU/GPU. But one may end up waiting in line. (The TPU-code may be commented.)**

**IDEAS TO IMPROVE:**
- Search for pretrained nets online. (There are many publications on this subject.)
- Use Graph Neural Networks
- Use other finger prints (e.g. 3D-finger prints)
- Build three independent models: one for each target protein. (May be much easier to learn separate interaction patterns.) (---> Already done! See "split_model()".)

**Questions:**
- How to set float64-policy?
- How to properly use tf.py_function
- How to properly use GPU?

## Switches:

In [68]:
# If submitting this file, different parameters will be used
submit = True

## Imports

In [69]:
import tensorflow as tf
import os
import numpy as np
import sys

# rdkit helps generating characteristics of molecules:
if submit:
    !pip install rdkit  --no-dependencies
else:
    # only install if not installed anymore
    if not os.path.isdir('/kaggle/working/mysitepackages'):
        # it was important to use "--no-dependencies" Otherwise submissions would not work anymore!!
        !pip install rdkit  --no-dependencies --target=/kaggle/working/mysitepackages
    sys.path.append('/kaggle/working/mysitepackages')
import rdkit
import rdkit.Chem as Chem
from rdkit.Chem import AllChem

gpu_name = tf.test.gpu_device_name()
if "GPU" not in gpu_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(gpu_name))

GPU device not found
Found GPU at: 


## Select Hyperparameters

In [70]:
# Parameters for testing (small numbers):

N_BITS_FINGERPRINT = 1024 # for ECFP-Fingerprint
N_RADIUS = 4 # for ECFP-Fingerprint
BATCH_SIZE = 25
N_TRAIN = 5000#-1 # set to -1 for "all"
N_TEST = 1000#-1 # set to -1 for "all"
N_EPOCHS = 15
WITH_DROPOUT = False
DROPOUT_RATE = 0.05
#ACTIVATION = 'relu'
ACTIVATION = tf.keras.layers.LeakyReLU(negative_slope=0.01)
HIDDEN_NEURONS = [500, 5] #Numbers of neurons per internal layer:

# Parameters for submissions (larger numbers):
if submit:
    BATCH_SIZE = 8000 # should be big, if (not RESAMPLE_INSTEAD_OF_REWEIGHT) in order to have at least some positive samples in the batch?
    N_BITS_FINGERPRINT = 2048 # for ECFP-Fingerprint
    N_RADIUS = 4 # for ECFP-Fingerprint
    N_TEST = -1 # -1 means "all"
    N_TRAIN = 5600000
    N_EPOCHS = 25
    HIDDEN_NEURONS = [251, 15]
    #ACTIVATION = 'relu'
    ACTIVATION = tf.keras.layers.LeakyReLU(negative_slope=0.01)
    WITH_DROPOUT = False
    DROPOUT_RATE = 0.005 # only needed if WITH_DROPOUT


In [71]:
train_path = '/kaggle/input/leash-BELKA/train.csv'
test_path = '/kaggle/input/leash-BELKA/test.csv'

In [72]:
## Old version (slow):
#def generate_ecfp(molecule, radius=N_RADIUS, bits=N_BITS_FINGERPRINT):
#    return list(AllChem.GetMorganFingerprintAsBitVect(molecule, radius, nBits=bits))

## New verion (3 times faster) (used in "Simple Net 2.ipynb"):
def ecfp(smile, radius=N_RADIUS, nBits=N_BITS_FINGERPRINT):
    """ creates a list containing the fingerprint
    accepts a bytes-string as smile
    """
    mol = Chem.MolFromSmiles(smile.numpy())
    fp_list = AllChem.GetMorganFingerprintAsBitVect(mol, radius, nBits=nBits).ToList()
    return fp_list

## Would it be more efficient to use bools? (But the following code throws an error.)
#def ecfp_bool(smile, radius=N_RADIUS, nBits=N_BITS_FINGERPRINT):
#    """ creates a tf.constant containing the fingerprint
#    """
#    ...
#    return tf.equals(..., 1)
#    )

## Functions for reading csv

In [73]:
@tf.py_function(Tout=tf.int32)
def get_fp(smile, radius=N_RADIUS, nBits=N_BITS_FINGERPRINT):
    fp_list = ecfp(smile, radius=N_RADIUS, nBits=N_BITS_FINGERPRINT)
    fp = tf.constant(fp_list, shape=(nBits,), dtype=tf.int32)
    return fp

def parse_fp_oneHot(x):
    oneHot = tf.math.equal(x['protein_name'], ['BRD4', 'sEH', 'HSA'])
    oneHot = tf.reshape(oneHot,(3,))
    fp = get_fp(x['molecule_smiles'])
    fp.set_shape((N_BITS_FINGERPRINT))
    return {'fp': fp, 'oneHot': oneHot}

def parse_fp_oneHot_binds(x,y):
    #tf.print("tf print ", y, type(y))
    #print("Normal  print ", y, type(y))
    y = tf.reshape(y, shape=(1,))
    y = tf.cast(y, tf.int32)
    #tf.print("2 tf print ", y, type(y))
    #print("2 Normal  print ", y, type(y))
    return parse_fp_oneHot(x), y



## Get Dataset:

In [74]:
def get_ds_csv(csv_path, batch_size, n_samples, labeled=True, repeat=False):
    """ n_samples=-1 means "take all"
    """
    if labeled:
        ds = tf.data.experimental.make_csv_dataset(
            csv_path,
            batch_size=4, # arbitrary .. ´will be overridden later on
            shuffle=False,
            num_epochs=1, # to prevent repeat()
            label_name='binds')
    else:
        ds = tf.data.experimental.make_csv_dataset(
            csv_path,
            batch_size=4,   # arbitrary .. ´will be overridden later on
            num_epochs=1, # to prevent repeat()
            shuffle=False)
    ds = ds.unbatch()
    if n_samples!=-1:
        ds = ds.take(n_samples)
    if labeled:
        ds = ds.map(parse_fp_oneHot_binds)
    else:
        ds = ds.map(parse_fp_oneHot)
    if repeat:
        ds = ds.repeat()
    ds = ds.batch(batch_size)
    return ds

ds = get_ds_csv(train_path, batch_size=BATCH_SIZE, n_samples=N_TRAIN, repeat=True)

#for elem in ds.take(1):
#    print(elem)
#    print()

## Callbacks

In [75]:
class TerminateAndBackup(tf.keras.callbacks.Callback):
    def __init__(self, 
                 directory="/kaggle/working/temp"):
        super().__init__()
        self.backup_file = directory+".weights.h5"
        try:
            os.stat(directory)
        except:
            os.mkdir(directory) 

    def on_epoch_begin(self, epoch, logs=None):
        #Save weights in the beginning of epoch
        self.model.save_weights(self.backup_file, overwrite=True)

    def on_train_batch_end(self, batch, logs=None):
        logs = logs or {}
        loss = logs.get('loss')
        if loss is not None:
            # Check if we have NaN or Inf
            if np.isnan(loss) or np.isinf(loss):
                print('Stopping learning --- ')
                model.load_weights(self.backup_file)
                self.model.stop_training  = True
terminate_and_backup = TerminateAndBackup()

reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='accuracy', factor=0.2,
                              patience=5, min_lr=0.001)

## Prepare training weights

In [76]:
# Calculate proportion of positive binds
# (Important for training weights)
if submit:
    n_consider = min(N_TRAIN, 500000)
else:
    n_consider = min(N_TRAIN, 30000)

initial_state = (0,0)
def combined_reduce(state, data):
    count, sum_binds = state
    count += 1
    sum_binds += int(data[1])
    return count, sum_binds
n_samples, n_binds = ds.unbatch().take(n_consider).reduce(initial_state, combined_reduce)
n_binds = n_binds.numpy()[0]
n_samples = n_samples.numpy()
print(f' {n_binds} of {n_samples} are positive')
binds_rate = n_binds / n_samples
print(f'bind rate = {binds_rate}')

 7 of 5000 are positive
bind rate = 0.0014


## Split-Model Topology

In [77]:
def splitted_model():
    ### creates single models for each target protein and concatenates them together
    
    def model_for_one_protein():
        fp_input = tf.keras.Input((N_BITS_FINGERPRINT,), name='fp')
        x = fp_input
        for N in HIDDEN_NEURONS:
            if WITH_DROPOUT:
                x = tf.keras.layers.Dropout(DROPOUT_RATE)(x)
            x = tf.keras.layers.Dense(N, activation=ACTIVATION)(x)
        output = tf.keras.layers.Dense(1)(x)
        return tf.keras.Model(fp_input, output)
            
    fp_input = tf.keras.Input((N_BITS_FINGERPRINT,), name='fp')
    protein_input = tf.keras.Input((3,), name='oneHot')
    model_BRD4 = model_for_one_protein()(fp_input)
    model_HSA  = model_for_one_protein()(fp_input)
    model_sEH  = model_for_one_protein()(fp_input)
    single_models = tf.keras.layers.Concatenate(axis=1)([model_BRD4, model_HSA, model_sEH])

    output = tf.keras.layers.Dot(axes=[1,1])([protein_input, single_models])
    model = tf.keras.Model(inputs=[fp_input, protein_input], outputs=output)
    
    # gamma=0 is just the normal BinaryCrossentropy. (But that one does not support training weights alpha.)
    loss_fn = tf.keras.losses.BinaryFocalCrossentropy(
        gamma=0, from_logits=True, apply_class_balancing=True, alpha=1-binds_rate)
    optimizer = tf.keras.optimizers.Adam(epsilon=1e-10)
    model.compile(optimizer="adam",
                  loss=loss_fn,
                  metrics=['accuracy'])
    return model

In [84]:
# if TPU is available:
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
#tf.tpu.experimental.initialize_tpu_system(tpu)
#tpu_strategy = tf.distribute.TPUStrategy(tpu)
#with tpu_strategy.scope():

#tf.keras.config.set_floatx('float64') # should use a "DType_Policy" instead

mirrored_strategy = tf.distribute.MirroredStrategy()
with mirrored_strategy.scope():
    #model = straight_model()
    model = splitted_model()
    #print(model.summary())
    model.fit(ds,
              epochs=N_EPOCHS,
              steps_per_epoch=N_TRAIN//BATCH_SIZE
              #,callbacks=[backup, terminate_on_nan]
              #,callbacks=[reduce_lr_on_nan]
              ,callbacks=[terminate_and_backup]
             )

Epoch 1/15
200/200 ━━━━━━━━━━━━━━━━━━━━ 11s 38ms/step - accuracy: 0.9981 - loss: 0.0106
Epoch 2/15
131/200 ━━━━━━━━━━━━━━━━━━━━ 2s 37ms/step - accuracy: 0.9961 - loss: 0.0032Stopping learning --- 
200/200 ━━━━━━━━━━━━━━━━━━━━ 5s 26ms/step - accuracy: 0.9946 - loss: nan 


## Predict on Test Set

In [79]:
print("Start predicting....")
test_ds = get_ds_csv(test_path, batch_size=215, n_samples=N_TEST, labeled=False)
y = model.predict(test_ds, verbose=0)
print('Done predicting')

Start predicting....
Done predicting


/opt/conda/lib/python3.10/contextlib.py:153: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


## Write Submission File

In [80]:
import pandas as pd
from scipy.special import expit, logit

yy = y[:,0]
d = pd.DataFrame({'id': range(len(yy)), 'binds': yy})
d['id'] = d['id'] + 295246830
d['binds'] = expit(d['binds'])
print("Start writing...")
d.to_csv('submission.csv', index=False, header=True)

print(d['binds'].describe())

Start writing...
count    1.000000e+03
mean     8.858638e-04
std      1.703754e-03
min      5.677725e-09
25%      3.669215e-06
50%      3.252820e-04
75%      1.128776e-03
max      2.256317e-02
Name: binds, dtype: float64


## Parquet does not seem to be faster:

In [81]:
#@tf.function # bringt es anscheinend nicht
#def _parse_parquet(ex):
#    oneHot = tf.math.equal(ex[b'protein_name'], ['BRD4', 'sEH', 'HSA'])
#    oneHot   = tf.reshape(oneHot,(3,))
#    fp = get_fp(ex[b'molecule_smiles'])
#    return {'fp': fp, 'oneHot': oneHot}

#import timeit
#import tensorflow_io as tfio
#ds_csv = test_ds
#ds_parquet = tfio.IODataset.from_parquet('/kaggle/input/leash-BELKA/test.parquet',columns=[b'molecule_smiles',b'protein_name'])
#ds_parquet = ds_parquet.map(_parse_parquet)
#print('CSV: ', timeit.timeit(lambda: model.predict(test_ds.unbatch().take(10000).batch(100)), number=1))
#print('Par: ', timeit.timeit(lambda: model.predict(ds_parquet.take(10000).batch(100)), number=1))

Warum ist es bei 10 nicht wirklich schneller, aber bei 10000 schon und warum bringt es bei 10 nichts, wenn der graph vorher erstellt wird?:

In [82]:
#import timeit
# TensorFlow imports
#from tensorflow.keras import Input, Model
#from tensorflow.keras.layers import Flatten, Dense
## Define the model (Inspired by mnist inputs)
#model = tf.keras.Sequential()
#model.add(tf.keras.Input(shape=(28,28,)))
#model.add(Flatten())
#model.add(Dense(256,"relu"))
#model.add(Dense(128,"relu"))
#model.add(Dense(256,"relu"))
#model.add(Dense(10,"softmax"))
## Dummy data with MNIST image sizes
#X = tf.random.uniform([1000, 28, 28])
## Eager Execution to do inference (Model untrained as we are evaluating speed of inference)
#eager_modell = model
#print("Eager time:", timeit.timeit(lambda: eager_modell(X,training=False), number=10))
#
##Graph Execution to do inference (Model untrained as we are evaluating speed of inference)
#graph_modell = tf.function(eager_modell) # Wrap the model with tf.function
#y = graph_model(X,training=False)
#print("Graph time:", timeit.timeit(lambda: graph_modell(X,training=False), number=10))